In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
from tqdm import tqdm
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Himanshu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data_location = "E:/datasets/word2vec/text8/text8"
window_size = 4

In [3]:
data = open(data_location, "rb").read()
data = data.decode("utf8")

In [4]:
token_index = {}
token_counts = {}

j=1
tokens = data.split(" ")[: 50000]
tokens = [token for token in tokens if token not in stopwords]
print(len(tokens))
for index, token in enumerate(tokens):
    if token in token_index:
        token_counts[token] += 1
    else:
        token_index[token] = j
        token_counts[token] = 1
        j += 1

30577


In [5]:
inputs = []
outputs = []
for i in range(len(tokens)):
    inputs.append(token_index[tokens[i]])
    window_start_index = i - window_size
    window_start_index = 0 if window_start_index < 0 else window_start_index
    window_end_index = i + window_size
    window_end_index = len(tokens) if window_end_index >= len(tokens) - 1 else window_end_index
    output = []
    for j in range(window_start_index, window_end_index):
        if j != i:
            output.append(token_index[tokens[j]])
    outputs.append(output)

In [6]:
class Skipgram:
    
    def __init__(self, num_features, vocab_size):
        self.num_features = num_features
        self.w1 = np.random.uniform(0.0, 0.5, size=(vocab_size, self.num_features))
        self.w2 = np.random.uniform(0.0, 0.5, size=(self.num_features, vocab_size))
        self.vocab_size = vocab_size
    
    def _forward_pass(self, model_input):
        self.h1 = np.dot(model_input, self.w1)
        self.h2 = np.dot(self.h1, self.w2)
        self.h2_exp = np.exp(self.h2 - np.max(self.h2))
        self.softmax_score = self.h2_exp/np.sum(self.h2_exp)
        return self.softmax_score
    
    def _backward_pass(self, model_input, model_output, lr):
        t_minus_s = (model_output - self.softmax_score)
        w2_grad = np.dot(np.reshape(self.h1, (self.num_features, 1)),
                         np.reshape(t_minus_s, (1, self.vocab_size)))
        w1_grad = np.dot(np.reshape(model_input, (self.vocab_size, 1)),
                         np.reshape(np.dot(t_minus_s, self.w1), (1, self.num_features)))
        self.w1 = self.w1 + (lr * w1_grad)
        self.w2 = self.w2 + (lr * w2_grad)
    
    def fit(self, model_inputs, model_outputs, lr, epochs):
        for epoch in range(epochs):
            loss = 0.0
            sample_indices = np.arange(len(model_inputs) - 1)
            np.random.shuffle(sample_indices)
            for i in tqdm(sample_indices):
                model_input = np.zeros((self.vocab_size))
                model_input[model_inputs[i]] = 1.0
                self._forward_pass(model_input=model_input)
                model_output = np.zeros((self.vocab_size))
                for j in model_outputs[i]:
                    model_output[j] += 1/len(model_outputs[i])
                self._backward_pass(model_input, model_output, lr)
                loss += -1 * np.log(np.dot(self.softmax_score, model_output))
            print("epochs: {0} loss: {1}".format(str(epoch+1), str(loss)))
            pickle.dump(self, open("skipgram_normal_epoch_{0}".format(str(epoch + 1)), "wb"))
            
    def most_similar(self, input_word, token_index):
        assert input_word in token_index
        word_vectors = (self.w1 + self.w2.transpose()) / 2
        prediction = cosine_similarity(word_vectors[token_index[input_word]].reshape(1, -1), word_vectors)[0]
        sorted_index = np.argsort(prediction)[::-1].tolist()
        rev_token_index = {j: i for i, j in token_index.items()}
        for index in range(10):
            print(rev_token_index[sorted_index[index]])

In [7]:
model = Skipgram(100, len(token_index) + 1)
model.fit(inputs, outputs, 0.1, 300)



  0%|                                                                                                                                      | 0/30576 [00:00<?, ?it/s]

epochs: 1 loss: 209012.98596704492


  0%|                                                                                                                                      | 0/30576 [00:00<?, ?it/s]

epochs: 2 loss: 201048.68955743808


  0%|                                                                                                                                      | 0/30576 [00:00<?, ?it/s]

...

epochs: 190 loss: 140235.81656063907


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30576/30576 [54:44<00:00,  9.31it/s]


epochs: 191 loss: 140304.42678658685


 52%|███████████████████████████████████████████████████████████████▍                                                          | 15887/30576 [27:41<06:20, 38.57it/s]

KeyboardInterrupt: 

In [12]:
sorted(token_counts.items(), key=lambda x: x[1])[::-1]

[('one', 931),
 ('nine', 481),
 ('zero', 444),
 ('two', 358),
 ('eight', 344),
 ('lincoln', 334),
 ('six', 273),
 ('three', 238),
 ('five', 204),
 ('four', 195),
 ('seven', 163),
 ('autism', 160),
 ('aristotle', 137),
 ('also', 123),
 ('rand', 108),
 ('war', 106),
 ('achilles', 102),
 ('anarchism', 102),
 ('many', 100),
 ('people', 88),
 ('autistic', 87),
 ('states', 84),
 ('would', 83),
 ('first', 78),
 ('anarchists', 76),
 ('may', 75),
 ('philosophy', 73),
 ('anarchist', 71),
 ('see', 69),
 ('time', 68),
 ('abraham', 67),
 ('state', 67),
 ('best', 64),
 ('often', 60),
 ('united', 59),
 ('others', 58),
 ('social', 58),
 ('altruism', 56),
 ('union', 56),
 ('however', 53),
 ('new', 52),
 ('american', 50),
 ('alabama', 48),
 ('slavery', 46),
 ('life', 46),
 ('general', 46),
 ('academy', 44),
 ('used', 43),
 ('abu', 42),
 ('film', 42),
 ('movement', 42),
 ('although', 42),
 ('early', 42),
 ('president', 41),
 ('dhabi', 41),
 ('form', 41),
 ('man', 41),
 ('history', 41),
 ('national', 40),

In [18]:
model.most_similar("autism", token_index)

autism
spectrum
autistic
disorder
asperger
syndrome
people
information
treatment
often
